# Portfolio Analysis with Statistics

To access stock data, we will use the [yfinance library](https://github.com/ranaroussi/yfinance). We first download the historical monthly stock prices for the chosen stocks/tickers(slightly modifying the code in the library tutorial). The data comes in the form of a pandas dataframe with multi-level headers, so we also unstack the levels for simpler access.

In [63]:
import yfinance as yf
import numpy as np
import scipy.stats as stats
import pandas as pd
ticks = ["AMD", "GE", "MSFT", "PILL", "TQQQ", "XLV"]
data = yf.download(tickers = ticks, interval = "1mo", group_by = 'ticker', auto_adjust = True, threads = True)

[*********************100%***********************]  6 of 6 completed


In [64]:
sp500_raw = yf.download(tickers = "^GSPC", interval = "1mo", group_by = 'ticker', auto_adjust = True, threads = True)

[*********************100%***********************]  1 of 1 completed


In [65]:
# unstack
stocks_raw = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
stocks_raw = stocks_raw.sort_values(by=['Date', 'Ticker'])
stocks_raw.tail(10)

,Ticker,Close,High,Low,Open,Volume
Date,,,,,,
2021-06-01,MSFT,265.019989,267.850006,243.000000,251.229996,447269400.0
2021-06-01,PILL,24.889999,25.799999,21.160000,22.400000,832800.0
2021-06-01,TQQQ,117.510002,119.489998,97.379997,103.849998,482349500.0
2021-06-01,XLV,125.211380,125.589908,120.141169,123.268963,175766000.0
2021-06-28,AMD,87.080002,88.000000,86.150002,86.379997,29748371.0
2021-06-28,GE,12.890000,13.160000,12.790000,13.160000,52940032.0
2021-06-28,MSFT,268.720001,268.899994,265.959991,266.184998,18592255.0
2021-06-28,PILL,24.980000,25.230000,24.775000,25.230000,8524.0
2021-06-28,TQQQ,121.669998,121.879997,118.750000,118.769997,22050626.0


Since we only want one stock price per month, we filter out the last row of each asset if it does not fall on the first day of the month. We also only keep the last 5 years of data to maintain an accurate representation of each company's relevant returns(profitability in the 1990s does not entail profitability in 2020s). Because we require the previous month's price to compute the current month's return, we need to keep an extra month(a total of 61 months)

In [66]:
recent = stocks_raw.index[-1] - pd.DateOffset(day = 1)
begin = recent - pd.DateOffset(years = 5) - pd.DateOffset(months = 1)
stocks = stocks_raw.loc[(stocks_raw.index <= recent) & (stocks_raw.index >= begin)].copy()
sp500 = sp500_raw.loc[(sp500_raw.index <= recent) & (sp500_raw.index >= begin)].copy()
stocks.head()

,Ticker,Close,High,Low,Open,Volume
Date,,,,,,
2016-05-01,AMD,4.570000,4.710000,3.450000,3.580000,394438600.0
2016-05-01,GE,26.446421,27.111298,25.422858,26.805104,633509240.0
2016-05-01,MSFT,48.487541,48.487541,45.248939,45.742963,530704800.0
2016-05-01,TQQQ,8.690357,8.731107,7.397199,7.749803,759457200.0
2016-05-01,XLV,65.184143,65.339450,62.680928,63.877719,211005900.0


To compute the (percent) return of a specified observation, we subtract the current price with last month's price and divide by last month's price. We can easily vectorize this by subtracting an array of the (open) prices without the last observation from an array of the (open) prices without the first observation. We then divide by the former.  

In [67]:
for t in ticks:
    # turn into np array to avoid indexing issues
    nofirst = np.array(stocks.loc[stocks['Ticker'] == t, 'Open'].iloc[1:])
    nolast = np.array(stocks.loc[stocks['Ticker'] == t, 'Open'].iloc[:len(stocks.loc[stocks['Ticker'] == t]) - 1])
    # add back index before assignment
    stocks.loc[stocks['Ticker'] == t, 'PercReturns'] = pd.Series((nofirst - nolast) / nolast, index = stocks.loc[stocks['Ticker'] == t, 'Open'].iloc[1:].index)

spnofirst = np.array(sp500['Open'].iloc[1:])
spnolast = np.array(sp500['Open'].iloc[:len(sp500) - 1])
spreturns = (spnofirst - spnolast) / spnolast

To make calculating parameters easier, we can pivot the dataframe such that each ticker's percent returns form individual columns. Note that we need to mask the data matrix to ignore NaN values. 

In [93]:
stockpivot = stocks.pivot(columns = 'Ticker', values = 'PercReturns').iloc[1:]
returnarr = stockpivot.to_numpy()
# mask to ignore NaN for relatively new stocks
returnmean = np.ma.masked_invalid(returnarr).mean(axis = 0).data
#returncov = np.ma.cov(np.ma.masked_invalid(returnarr), rowvar = False).data
#invcov = np.linalg.lstsq(a = returncov, b = np.eye(len(ticks)), rcond = None)[0]

In [112]:
betas = np.zeros(len(ticks))
alphas = np.zeros(len(ticks))
unsyserr = np.zeros(len(ticks))
for i in np.arange(len(ticks)):
    treturn = returnarr[:,i]
    tnonan = treturn[np.logical_not(np.isnan(treturn))]
    spmatch = 
    betas[i], alphas[i], r, p, se = stats.linregress(spreturns[(len(spreturns) - len(tnonan)):], tnonan)
    unsyserr[i] = np.sum((returnarr[:,i] - alphas[i] - betas[i]*spreturns)**2) / (len(spreturns) - 2)

ValueError: operands could not be broadcast together with shapes (42,) (61,) 

In [107]:
simdf = pd.DataFrame(data = {'alpha': alphas, 'beta': betas, 'eps': unsyserr, 'rmean': returnmean}, index = stockpivot.columns.values)
simdf['excess'] = simdf['rmean'] / simdf['beta']
simdf = simdf.sort_values(by=['excess'], ascending = False)

In [108]:
num = simdf['rmean'] * simdf['beta'] / simdf['eps']
den = simdf['beta']**2 / simdf['eps']
simdf['C'] = spreturns.var() * num.cumsum() / (1 + spreturns.var() * den.cumsum())

In [109]:
cutoff = simdf.loc[simdf['C'] < simdf['excess']]

In [110]:
z = (cutoff['beta'] / cutoff['eps']) * (cutoff['excess'] - cutoff['C'])
z / z.sum()

MSFT    0.915554
AMD     0.084446
dtype: float64

In [111]:
simdf

,alpha,beta,eps,rmean,excess,C
MSFT,0.019163,0.806234,0.001098,0.029551,0.036653,0.020822
AMD,0.040550,1.936114,0.019882,0.065494,0.033828,0.022814
TQQQ,0.015375,3.314037,0.003876,0.058072,0.017523,0.019125
XLV,0.001822,0.771605,0.000569,0.011763,0.015245,0.018331
PILL,-0.019008,2.904259,NaN,0.018033,0.006209,NaN
GE,-0.018372,1.161444,0.011282,-0.003408,-0.002934,0.017845
